# Example of wrapping CRPS for ensembles from `scoringrules` in `scores`

In [1]:
import scoringrules
import xarray as xr
import numpy as np

from scores.probability import crps_for_ensemble
import scores

from typing import Literal, Optional, Sequence

In [2]:
fcst = xr.DataArray(
    data=[[0.0, 4, 3, 7], [1, -1, 2, 4], [0, 1, 4, np.nan], [2, 3, 4, 1], [2, np.nan, np.nan, np.nan]],
    dims=["stn", "ens_member"],
    coords={"stn": [101, 102, 103, 104, 105], "ens_member": [1, 2, 3, 4]},
)
obs = xr.DataArray(data=[2.0, 3, 1, np.nan, 4], dims=["stn"], coords={"stn": [101, 102, 103, 104, 105]})

In [3]:
def crps_ensemble_wrapper(
        fcst: xr.DataArray, 
        obs: xr.DataArray, 
        ensemble_member_dim: str, 
        *, 
        method: Literal["ecdf", "fair", "pwm"]="nrg", 
        reduce_dims: Optional[Sequence[str]] = None, 
        preserve_dims: Optional[Sequence[str]] = None, 
        weights: Optional[xr.DataArray] = None):
    """
    CRPS for ensembles wrapper around scoringrules
    """

    weights_dims = None
    if weights is not None:
        weights_dims = weights.dims

    dims_for_mean = scores.utils.gather_dimensions(
        fcst.dims,
        obs.dims,
        weights_dims=weights_dims,
        reduce_dims=reduce_dims,
        preserve_dims=preserve_dims,
        score_specific_fcst_dims=ensemble_member_dim,
    )

    if method == "ecdf":
        method="nrg"
    
    # Put the ensemble member dimension last to align with the behaviour of scoringrules
    fcst = fcst.transpose(..., ensemble_member_dim)

    result = xr.apply_ufunc(
        scoringrules.crps_ensemble, 
        obs, 
        fcst, 
        kwargs={"estimator": method}, 
        input_core_dims=[obs.dims, fcst.dims],
        output_core_dims=[obs.dims],
        dask="parallelized"
    )
    
    # Apply weights and take mean if required
    result = scores.functions.apply_weights(result, weights=weights)
    result = result.mean(dim=dims_for_mean) 

    return result

## Compare emperical (nrg) CRPS versions

In [4]:
# scoringrules
crps_ensemble_wrapper(fcst, obs, ensemble_member_dim="ens_member", preserve_dims="all")

<xarray.DataArray (stn: 5)>
array([1.125, 1.   ,   nan,   nan,   nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [14]:
# check that reordering the dimesnions of fcst still produces the right output with scoringrules
crps_ensemble_wrapper(fcst.T, obs, ensemble_member_dim="ens_member", preserve_dims="all")

<xarray.DataArray (stn: 5)>
array([1.125, 1.   ,   nan,   nan,   nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [5]:
# scores
crps_for_ensemble(fcst, obs, ensemble_member_dim="ens_member", preserve_dims="all")

<xarray.DataArray (stn: 5)>
array([1.125     , 1.        , 0.44444444,        nan, 2.        ])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

Note the difference in handling NaNs

In [6]:
fcst.sel(stn=103)

<xarray.DataArray (ens_member: 4)>
array([ 0.,  1.,  4., nan])
Coordinates:
    stn         int64 103
  * ens_member  (ens_member) int64 1 2 3 4

## Test that it works with Dask

In [7]:
fcst_dask = fcst.chunk()
obs_dask = obs.chunk()

In [8]:
dask_result = crps_ensemble_wrapper(fcst_dask, obs_dask, ensemble_member_dim="ens_member", preserve_dims="all")
dask_result

<xarray.DataArray (stn: 5)>
dask.array<transpose, shape=(5,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [9]:
dask_result.compute()

<xarray.DataArray (stn: 5)>
array([1.125, 1.   ,   nan,   nan,   nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

## Check Fair CRPS

In [10]:
# scoringrules
crps_ensemble_wrapper(fcst, obs, ensemble_member_dim="ens_member", preserve_dims="all", method="fair")

<xarray.DataArray (stn: 5)>
array([0.66666667, 0.66666667,        nan,        nan,        nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [11]:
# scores
crps_for_ensemble(fcst, obs, ensemble_member_dim="ens_member", preserve_dims="all", method="fair")

<xarray.DataArray (stn: 5)>
array([0.66666667, 0.66666667, 0.        ,        nan,        nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

## twCRPS for ensembles

In [12]:
def twcrps_ensemble_wrapper(
        fcst: xr.DataArray, 
        obs: xr.DataArray, 
        ensemble_member_dim: str, 
        v_func,
        *, 
        method: Literal["ecdf", "fair", "pwm"]="nrg", 
        reduce_dims: Optional[Sequence[str]] = None, 
        preserve_dims: Optional[Sequence[str]] = None, 
        weights: Optional[xr.DataArray] = None):
    """
    CRPS for ensembles wrapper around scoringrules
    """

    weights_dims = None
    if weights is not None:
        weights_dims = weights.dims

    dims_for_mean = scores.utils.gather_dimensions(
        fcst.dims,
        obs.dims,
        weights_dims=weights_dims,
        reduce_dims=reduce_dims,
        preserve_dims=preserve_dims,
        score_specific_fcst_dims=ensemble_member_dim,
    )

    if method == "ecdf":
        method="nrg"
    
    # Put the ensemble member dimension last to align with the behaviour of scoringrules
    fcst = fcst.transpose(..., ensemble_member_dim)

    result = xr.apply_ufunc(
        scoringrules.twcrps_ensemble, 
        obs, 
        fcst, 
        v_func,
        kwargs={"estimator": method}, 
        input_core_dims=[obs.dims, fcst.dims, []],
        output_core_dims=[obs.dims],
        dask="parallelized"
    )
    
    # Apply weights and take mean if required
    result = scores.functions.apply_weights(result, weights=weights)
    result = result.mean(dim=dims_for_mean) 

    return result

In [13]:
def v_func(x):
    return np.maximum(x, 0)

twcrps_ensemble_wrapper(fcst, obs, ensemble_member_dim="ens_member", v_func=v_func, preserve_dims="all")

<xarray.DataArray (stn: 5)>
array([1.125 , 0.9375,    nan,    nan,    nan])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105